<b>Part 1 - Build dataframe of toronto neighborhoods</B>

In [1]:
!conda install -c conda-forge geocoder geopy folium beautifulsoup4 --yes

import numpy as np
import pandas as pd
import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.



In [2]:
sourcedata = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(sourcedata, 'html.parser')
postalCodes = []
boroughs = []
hoods = []
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodes.append(cells[0].text)
        boroughs.append(cells[1].text)
        hoods.append(cells[2].text.rstrip('\n'))

In [3]:
temp = { 'PostalCode': postalCodes,
        'Borough': boroughs,
        'Neighborhood': hoods}
postal_codes_df = pd.DataFrame.from_dict(temp)
postal_codes_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
postal_codes_df.replace("Not assigned", np.nan, inplace=True)
postal_codes_df.dropna(subset=['Borough'], inplace=True)
postal_codes_df['Neighborhood'].fillna(postal_codes_df['Borough'], inplace=True)

In [5]:
postal_codes_df = postal_codes_df.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
postal_codes_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
postal_codes_df.shape

(103, 3)

<b>Part 2 - Add neighborhood coordinates using csv file</b>

In [7]:
coords_df = pd.read_csv("https://cocl.us/Geospatial_data")
coords_df.rename(columns={'Postal Code':'PostalCode'}, inplace = True)
coords_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
tempdf1 = postal_codes_df.set_index('PostalCode')
tempdf2 = coords_df.set_index('PostalCode')
toronto_coords_df = pd.concat([tempdf1, tempdf2], axis=1, join='inner')
toronto_coords_df.reset_index(inplace=True)
toronto_coords_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<b>Part 3 - Explore the neighborhoods in Toronto</b>

In [9]:
from pandas.io.json import json_normalize
import json
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim
import geocoder

from sklearn.cluster import KMeans

In [10]:
g = geocoder.arcgis('Toronto Ontario')
toronto_lat = g.latlng[0]
toronto_lng = g.latlng[1]

map_tr = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=11)

for lat, long, post, borough, hood in zip(toronto_coords_df['Latitude'], toronto_coords_df['Longitude'],toronto_coords_df['PostalCode'], toronto_coords_df['Borough'],toronto_coords_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, hood)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tr)
map_tr